In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


plt.style.use('ggplot')

import nltk

In [7]:
df = pd.read_csv('../../data/p00_tweets/processedNegative.csv')

In [11]:
df.head()

,How unhappy some dogs like it though,talking to my over driver about where I'm goinghe said he'd love to go to New York too but since Trump it's probably not,Does anybody know if the Rand's likely to fall against the dollar? I got some money I need to change into R but it keeps getting stronger unhappy,I miss going to gigs in Liverpool unhappy,There isnt a new Riverdale tonight ? unhappy,it's that A*dy guy from pop Asia and then the translator so they'll probs go with them around Aus unhappy,Who's that chair you're sitting in? Is this how I find out. Everyone knows now. You've shamed me in pu,don't like how jittery caffeine makes me sad,My area's not on the list unhappy think I'll go LibDems anyway,I want fun plans this weekend unhappy,...,and yet if parents invest in child's emotional education by taking child out of school on holiday early that's un,YG should have sent them to MCD. I want to see them holding the trophy unhappy anyways .9,i want more orientation unhappy,unhappy they not,YG should have sent them to MCD. I want to see them holding the trophy unhappy anyways .10,wish knock out lang talaga for the new school year are good and cooperative groupmates please unhappy,i miss so much unhappy,Same unhappy .1,Hi instant message your friend friend lang,hindi close friend? unhappy
